In [72]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import ExtraTreesClassifier

In [73]:
train = pd.read_csv('open/train.csv').drop(columns=['ID'])
test = pd.read_csv('open/test.csv').drop(columns=['ID'])

In [74]:
X = train.drop('임신 성공 여부', axis=1)
y = train['임신 성공 여부']
X = X.drop(columns=['임신 시도 또는 마지막 임신 경과 연수', '착상 전 유전 검사 사용 여부', 'PGD 시술 여부','PGS 시술 여부','난자 해동 경과일','배아 해동 경과일'])
test = test.drop(columns=['임신 시도 또는 마지막 임신 경과 연수', '착상 전 유전 검사 사용 여부', 'PGD 시술 여부','PGS 시술 여부','난자 해동 경과일','배아 해동 경과일'])
#임신 시도 또는 마지막 임신 경과 연수 열 삭제
#착상 전 유전 검사 사용 여부 삭제
#PGD 시술 여부 삭제
#PGS 시술 여부 삭제
#난자 해동 경과일 삭제
#배아 해동 경과일 삭제

In [75]:
categorical_columns = [
    "시술 시기 코드",
    "시술 당시 나이",
    "시술 유형",
    "특정 시술 유형",
    "배란 자극 여부",
    "배란 유도 유형",
    "단일 배아 이식 여부",
    "착상 전 유전 진단 사용 여부",
    "남성 주 불임 원인",
    "남성 부 불임 원인",
    "여성 주 불임 원인",
    "여성 부 불임 원인",
    "부부 주 불임 원인",
    "부부 부 불임 원인",
    "불명확 불임 원인",
    "불임 원인 - 난관 질환",
    "불임 원인 - 남성 요인",
    "불임 원인 - 배란 장애",
    "불임 원인 - 여성 요인",
    "불임 원인 - 자궁경부 문제",
    "불임 원인 - 자궁내막증",
    "불임 원인 - 정자 농도",
    "불임 원인 - 정자 면역학적 요인",
    "불임 원인 - 정자 운동성",
    "불임 원인 - 정자 형태",
    "배아 생성 주요 이유",
    "총 시술 횟수",
    "클리닉 내 총 시술 횟수",
    "IVF 시술 횟수",
    "DI 시술 횟수",
    "총 임신 횟수",
    "IVF 임신 횟수",
    "DI 임신 횟수",
    "총 출산 횟수",
    "IVF 출산 횟수",
    "DI 출산 횟수",
    "난자 출처",
    "정자 출처",
    "난자 기증자 나이",
    "정자 기증자 나이",
    "동결 배아 사용 여부",
    "신선 배아 사용 여부",
    "기증 배아 사용 여부",
    "대리모 여부"
]

# One-Hot Encoder 초기화
onehot_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

# 인코딩 적용
X_train_encoded = onehot_encoder.fit_transform(X[categorical_columns])
X_test_encoded = onehot_encoder.transform(test[categorical_columns])

# 인코딩된 데이터를 데이터프레임으로 변환
X_train_encoded = pd.DataFrame(X_train_encoded, columns=onehot_encoder.get_feature_names_out(categorical_columns))
X_test_encoded = pd.DataFrame(X_test_encoded, columns=onehot_encoder.get_feature_names_out(categorical_columns))

# 기존 데이터프레임과 결합
X_train_encoded = pd.concat([X.drop(columns=categorical_columns).reset_index(drop=True), X_train_encoded], axis=1)
X_test_encoded = pd.concat([test.drop(columns=categorical_columns).reset_index(drop=True), X_test_encoded], axis=1)

In [76]:
numeric_columns = [
    "총 생성 배아 수",
    "미세주입된 난자 수",
    "미세주입에서 생성된 배아 수",
    "이식된 배아 수",
    "미세주입 배아 이식 수",
    "저장된 배아 수",
    "미세주입 후 저장된 배아 수",
    "해동된 배아 수",
    "해동 난자 수",
    "수집된 신선 난자 수",
    "저장된 신선 난자 수",
    "혼합된 난자 수",
    "파트너 정자와 혼합된 난자 수",
    "기증자 정자와 혼합된 난자 수",
    "난자 채취 경과일",
    "난자 혼합 경과일",
    "배아 이식 경과일",
]

X_train_encoded[numeric_columns] = X_train_encoded[numeric_columns].fillna(0)
X_test_encoded[numeric_columns] = X_test_encoded[numeric_columns].fillna(0)
y=y.fillna(0)

In [77]:
model = ExtraTreesClassifier(random_state=101)

model.fit(X_train_encoded, y)

ExtraTreesClassifier(random_state=42)

In [78]:
pred_proba = model.predict_proba(X_test_encoded)[:, 1]

sample_submission = pd.read_csv('open/sample_submission.csv')
sample_submission['probability'] = pred_proba

sample_submission.to_csv('./baseline_submit.csv', index=False)

In [79]:
X_train_encoded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3190 entries, 0 to 3189
Columns: 178 entries, 총 생성 배아 수 to 대리모 여부_nan
dtypes: float64(178)
memory usage: 4.3 MB
